In [16]:
# statistic n_tweets_in_topic
import numpy as np

topic_result_path = "./output/topic_result_2023-04-22-13-40-46_1iter.txt"
reader = open(topic_result_path, "r")

n_topic = 20
n_tweets_in_topic = np.zeros((n_topic))
for line in reader.readlines():
    split_line = line.split(" ")
    for i in range(len(split_line)):
        if split_line[i] == "(topic:":
            n_tweets_in_topic[int(split_line[i + 1].split(')')[0])] += 1
            
n_tweets_in_topic

array([15128.,  6969.,  9000., 11717.,  7985.,  6481.,  7815., 13763.,
       11011.,  6301., 11930.,  8227.,  9688.,  9730.,  6083.,  5855.,
        7523., 11941., 10667.,  6135.])

In [7]:
import pandas as pd

# Read the CSV file
"""
tweets - '.csv' organization of the original corpus
tweets_btm - Topic model text file after obtaining the topic classification results, organized as "document (Topic: 8)"
"""
after_preprocess_dataset_path = "./data/after_preprocess_dataset_clean_english_only_new.csv"
topic_result_path = "./output/topic_result_2023-04-22-13-40-46_1iter.txt"

tweets = pd.read_csv(after_preprocess_dataset_path)
tweets_btm = open(topic_result_path).read().splitlines()
tweets.head(5), tweets_btm[:5], tweets.shape, len(tweets_btm)

(                                          text_clean
 0  ai take job within five year expert warn exper...
 1  although chatgpt rapidli gain popular also bec...
 2  amplifi human potenti school educ board readi ...
 3  analyst eran shimoni omer tsarfati detail crea...
 4  artificialintellig ai take job within five yea...,
 ['ai take job within five year expert warn expert explain bot like chatgpt domin labor market via  (topic: 7)',
  'although chatgpt rapidli gain popular also becom issu concern openaichatgpt ai nlp  (topic: 17)',
  'amplifi human potenti school educ board readi aiin aiineduc chatgpt educ  (topic: 3)',
  'analyst eran shimoni omer tsarfati detail creat polymorph malwar use chatgpt plan releas work learn purpos  (topic: 13)',
  'artificialintellig ai take job within five year expert explain bot like chatgpt domin labor market uselesseat take vax  (topic: 7)'],
 (183949, 1),
 183949)

In [8]:
tweets.rename(columns={"text_clean": "text"}, inplace=True) # Change the table header to 'text'
tweets.head(5)

,text
0,ai take job within five year expert warn exper...
1,although chatgpt rapidli gain popular also bec...
2,amplifi human potenti school educ board readi ...
3,analyst eran shimoni omer tsarfati detail crea...
4,artificialintellig ai take job within five yea...


In [9]:
import pickle
import numpy as np

# Load the BTM model file
model_path = "./models/btm_model_2023-04-22-13-40-46_1iter.pkl"
f = open(model_path,'rb')
biterm_model = pickle.load(f)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from operator import itemgetter

# vectorize texts
vec = CountVectorizer(stop_words='english')
tweets_list = [i for item in tweets.values for i in item]
X = vec.fit_transform(tweets_list).toarray()

# vocab - Get all words
vocab = np.array([t for t, i in sorted(vec.vocabulary_.items(),
                                     key=itemgetter(1))])

# The top probability list of the most likely words for each topic
topic_top_prob = biterm_model.phi_wz # phi_wz: [word, topic] Word distribution probability on each topic

# Find the topM words for each topic
def generate_topic_top_word(topic_top_prob, V, M = 10):
    """
    Args:
        topic_top_prob - The top probability list of the most likely words for each topic
        V [List] - A list of all the words
        M - Take the first M words
    Returns:
        topic_top_word Dict(List[Tuple()]) - The names of the top M words in probability
    """
    topic_top_word = dict()
    for z, P_wzi in enumerate(topic_top_prob.T): 
        """
            z - z-th topic
            P_wzi - The probability distribution of all words on the z-th topic
        """
        topic_top_word[z] = [] # Each topic consists of multiple tuples (word, prob)
        V_z_prob = np.sort(P_wzi)[:-(M + 1):-1] # Sort the probability distribution
        V_z = np.argsort(P_wzi)[:-(M + 1):-1] # Sort the probability distribution and find the index of the top words
        W_z = V[V_z] # Find the name of the word at the top of the list
        for prob, word in zip(V_z_prob, W_z): # Form the innermost tuple, meaning tuple(word, prob).
            topic_top_word[z].append((word, prob))
    return topic_top_word

topic_all_words = []
# top-M words for each topic
topic_top_word = generate_topic_top_word(topic_top_prob, vocab, 20)
for i in range(20):
    print_str = ""
    # print_str = f"topic {i}"
    for j in range(len(topic_top_word[i])):
        print_str += f"{topic_top_word[i][j][0]},"
    topic_all_words.append(print_str)
    print(print_str)

chatgpt,work,ai,use,job,write,openai,need,ask,human,tool,gpt,gener,new,help,like,industri,concern,creat,skill,
chatgpt,ai,work,like,use,gpt,look,job,team,ask,time,tool,educ,openai,industri,research,write,gener,skill,task,
chatgpt,ai,work,use,like,job,help,task,educ,make,need,assist,gener,model,new,content,way,openai,time,tool,
chatgpt,ai,work,educ,use,job,write,think,intellig,gener,artifici,tool,thing,school,new,student,human,technolog,chatbot,need,
chatgpt,ai,work,job,use,write,like,gpt,make,gener,tool,industri,train,peopl,help,good,new,data,way,human,
chatgpt,ai,job,work,use,like,make,dont,task,ask,think,data,futur,industri,human,peopl,train,better,languag,skill,
chatgpt,ai,job,work,use,new,like,task,learn,ask,data,help,futur,make,question,tool,technolog,gpt,think,busi,
chatgpt,ai,use,work,like,job,creat,googl,gener,make,task,new,train,write,educ,help,answer,model,time,prompt,
chatgpt,ai,work,use,time,job,like,think,human,gpt,team,tech,ask,peopl,make,technolog,educ,tri,code,world,
ch

['chatgpt,work,ai,use,job,write,openai,need,ask,human,tool,gpt,gener,new,help,like,industri,concern,creat,skill,',
 'chatgpt,ai,work,like,use,gpt,look,job,team,ask,time,tool,educ,openai,industri,research,write,gener,skill,task,',
 'chatgpt,ai,work,use,like,job,help,task,educ,make,need,assist,gener,model,new,content,way,openai,time,tool,',
 'chatgpt,ai,work,educ,use,job,write,think,intellig,gener,artifici,tool,thing,school,new,student,human,technolog,chatbot,need,',
 'chatgpt,ai,work,job,use,write,like,gpt,make,gener,tool,industri,train,peopl,help,good,new,data,way,human,',
 'chatgpt,ai,job,work,use,like,make,dont,task,ask,think,data,futur,industri,human,peopl,train,better,languag,skill,',
 'chatgpt,ai,job,work,use,new,like,task,learn,ask,data,help,futur,make,question,tool,technolog,gpt,think,busi,',
 'chatgpt,ai,use,work,like,job,creat,googl,gener,make,task,new,train,write,educ,help,answer,model,time,prompt,',
 'chatgpt,ai,work,use,time,job,like,think,human,gpt,team,tech,ask,peopl,make

In [26]:
import pandas as pd
df = pd.DataFrame()

topic_result_path = "./output/topic_result_2023-04-22-13-40-46_1iter.txt"
reader = open(topic_result_path, "r")
i = 0
for line in reader.readlines():
    if i % 1000 == 0:
        print(f"finished {i}/{183949}")
    row = []

    split_line = line.split("(")
    doc = split_line[0]
    topic_id = int(split_line[1].split(" ")[1].split(')')[0])
    row.append(f"topic{topic_id}")
    row.append(topic_all_words[topic_id])
    row.append(doc)
    temp_df = pd.DataFrame([row], columns=['topic', 'keywords', 'tweet'])
    df = pd.concat([df, temp_df], ignore_index=True)
    i += 1
df

finished 0/183949
finished 1000/183949
finished 2000/183949
finished 3000/183949
finished 4000/183949
finished 5000/183949
finished 6000/183949
finished 7000/183949
finished 8000/183949
finished 9000/183949
finished 10000/183949
finished 11000/183949
finished 12000/183949
finished 13000/183949
finished 14000/183949
finished 15000/183949
finished 16000/183949
finished 17000/183949
finished 18000/183949
finished 19000/183949
finished 20000/183949
finished 21000/183949
finished 22000/183949
finished 23000/183949
finished 24000/183949
finished 25000/183949
finished 26000/183949
finished 27000/183949
finished 28000/183949
finished 29000/183949
finished 30000/183949
finished 31000/183949
finished 32000/183949
finished 33000/183949
finished 34000/183949
finished 35000/183949
finished 36000/183949
finished 37000/183949
finished 38000/183949
finished 39000/183949
finished 40000/183949
finished 41000/183949
finished 42000/183949
finished 43000/183949
finished 44000/183949
finished 45000/183949
f

,topic,keywords,tweet
0,topic7,"chatgpt,ai,use,work,like,job,creat,googl,gener...",ai take job within five year expert warn exper...
1,topic17,"chatgpt,ai,work,use,job,write,educ,like,gener,...",although chatgpt rapidli gain popular also bec...
2,topic3,"chatgpt,ai,work,educ,use,job,write,think,intel...",amplifi human potenti school educ board readi ...
3,topic13,"chatgpt,ai,use,job,work,like,make,tool,task,he...",analyst eran shimoni omer tsarfati detail crea...
4,topic7,"chatgpt,ai,use,work,like,job,creat,googl,gener...",artificialintellig ai take job within five yea...
...,...,...,...
183944,topic2,"chatgpt,ai,work,use,like,job,help,task,educ,ma...",zzzz build high valu brand reason presenc tool...
183945,topic0,"chatgpt,work,ai,use,job,write,openai,need,ask,...",zzzz nice work curiou plan adjust chatgpt star...
183946,topic10,"chatgpt,ai,work,job,use,like,openai,gener,way,...",zzzz year ai autom busi save compani hr week b...
183947,topic6,"chatgpt,ai,job,work,use,new,like,task,learn,as...",zzzzzz content book buy find digit copi guaran...


In [27]:
save_table_path='./output/tweets_by_topic.csv'

df.to_csv(save_table_path, sep=',', index=False,header=True)